In [1]:
import igraph as ig
import numpy as np
import pandas as pd
from sklearn.metrics import adjusted_mutual_info_score as AMI

In [2]:
datadir='../Datasets/'

### Problem 1

Run various clustering algorithms (ECG, Louvain, Infomap, Label Propagation, Girvan-Newman, and CNM) for the karate club graph. For each algorithm tested, compare the partition found by the algorithm with the ground-truth (two communities) by computing the AMI score

In [3]:
z = ig.Graph.Famous('zachary')
z.vs['comm'] = [0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1]

In [4]:
# copied GitHub code to add ECG to the choice of community algorithms
def community_ecg(self, weights=None, ens_size=16, min_weight=0.05):
    W = [0]*self.ecount()
    ## Ensemble of level-1 Louvain 
    for i in range(ens_size):
        p = np.random.permutation(self.vcount()).tolist()
        g = self.permute_vertices(p)
        l = g.community_multilevel(weights=weights, return_levels=True)[0].membership
        b = [l[p[x.tuple[0]]]==l[p[x.tuple[1]]] for x in self.es]
        W = [W[i]+b[i] for i in range(len(W))]
    W = [min_weight + (1-min_weight)*W[i]/ens_size for i in range(len(W))]
    ## Force min_weight outside 2-core
    core = self.shell_index()
    ecore = [min(core[x.tuple[0]],core[x.tuple[1]]) for x in self.es]
    w = [W[i] if ecore[i]>1 else min_weight for i in range(len(ecore))]
    part = self.community_multilevel(weights=w)
    part.W = w
    part.CSI = 1-2*np.sum([min(1-i,i) for i in w])/len(w)
    return part
ig.Graph.community_ecg = community_ecg

ECG:

In [5]:
ecg = z.community_ecg().membership
print('AMI:',AMI(z.vs['comm'],ecg))

AMI: 0.5805154226518418


Louvain:

In [6]:
louvain = z.community_multilevel().membership
print('AMI:',AMI(z.vs['comm'],louvain))

AMI: 0.5653497612707893


Infomap:

In [7]:
infomap = z.community_infomap().membership
print('AMI:',AMI(z.vs['comm'],infomap))

AMI: 0.687422347904921


Label Propagation:

In [8]:
lp = z.community_label_propagation().membership
print('AMI:',AMI(z.vs['comm'],lp))

AMI: 0.5443932646993975


Girvan-Newman

In [9]:
gn = z.community_edge_betweenness()
print('AMI:',AMI(z.vs['comm'],gn.as_clustering(n=2).membership))

AMI: 0.8327564079186137


CNM:

In [10]:
cnm = z.community_fastgreedy()
print('AMI:',AMI(z.vs['comm'],cnm.as_clustering(n=2).membership))

AMI: 0.8334659946350965


### Problem 2

Run various clustering algorithms (ECG, Louvain, Infomap, Label Propagation, Girvan-Newman and CNM) on the GitHub ml graph. Which algorithms produce similar results? In order to answer this question, for each pair of algorithms, find the AMI score between the two results. 

In [11]:
## read edges and make graph
github_df = pd.read_csv(datadir+'GitHubDevelopers/musae_git_edges.csv')
gh = ig.Graph.DataFrame(github_df, directed=False)

## read node features
X = pd.read_csv(datadir+'GitHubDevelopers/musae_git_target.csv')

## map node names in edgelist to indices in the graph
idx = [int(i) for i in gh.vs['name']]
sorterIndex = dict(zip(idx,range(len(idx))))
X['Rank'] = X['id'].map(sorterIndex) 
X.sort_values(['Rank'], ascending=[True],inplace=True)
X.dropna(inplace=True)

lbl = ['web','ml']     ## node labels
gh.vs['lbl'] = [lbl[i] for i in list(X['ml_target'])]

## build the mlsubgraph
gh_ml = gh.subgraph([v for v in gh.vs() if v['lbl']=='ml'])

In [13]:
ecg = gh_ml.community_ecg().membership
louvain = gh_ml.community_multilevel().membership
infomap = gh_ml.community_infomap().membership
lp = gh_ml.community_label_propagation().membership

In [16]:
cnm = gh_ml.community_fastgreedy()
clusters = np.arange(5, 10)
modularity = np.zeros(len(clusters))
for c in range(len(clusters)):
    modularity[c] = gh_ml.modularity(cnm.as_clustering(n=clusters[c]))
plt.plot(clusters, modularity)
plt.xlabel('Number of clusters')
plt.ylabel('Modularity')
plt.show()
#.as_clustering(n=2).membership

InternalError: Error at src/community/community_misc.c:115: Number of steps is greater than number of rows in merges matrix: found 9734 steps, 7273 rows. -- Invalid value

In [17]:
gh_ml.is_simple()

True

In [ ]:
## the Girvan-Newman was not included as it was too slow
gn = gh_ml.community_edge_betweenness()

In [20]:
clustering_algorithms = [ecg, louvain, infomap, lp, cnm]
names = ['ECG', 'Louvain', 'Infomap', 'Label Propagation', 'CNM']
for i in range(len(clustering_algorithms)):
    for j in range(i):
        print('AMI between {0} and {1} algorithms: {2}'.format(names[i], names[j], AMI(clustering_algorithms[i],clustering_algorithms[j])))

AMI between Louvain and ECG algorithms: 0.5941542935296517
AMI between Infomap and ECG algorithms: 0.5758692329685827
AMI between Infomap and Louvain algorithms: 0.49208137321052553
AMI between Label Propagation and ECG algorithms: 0.4917535043937392
AMI between Label Propagation and Louvain algorithms: 0.5798755606421877
AMI between Label Propagation and Infomap algorithms: 0.29612211530266497


TypeError: len() of unsized object

### Problem 5

Take the ABCD graph we used to test quality measures. Check node roles. Compute how many nodes we have in each family (recall there are 4 families of non-hubs and 3 families of hubs). Plot the $(z(v), p(v))$ scores for all nodes as we did in Figure 5.3 for the karate club graph. 

### Problem 6

Compare time complexities of various clustering algorithms (ECG, Louvain, Infomap, Label Propagation, Girvan-Newman, CNM) using an ABCD synthetic graph with differet number of nodes, say, $n=100, 200, 400, 800, 1600, ...$. Which algorithm is the slowest, which one is the fastest?